In [1]:
import os
import ffm
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score

In [2]:
user_id_list = []
for i in range(57159, 94251):
    user_id_list.append(i)

# Prepare user features

In [25]:
month_list = ['05', '06','07', '08']
def get_user_features(directory):
    user_action_features = {}
    action_feature = {}
    user_title_features = {}
    title_feature = {}
    user_date_features = {}
    date_feature = {}
    user_online_frequency_count = {}
    date_hour_set = []
    title_set = []
    last_date = ""
    for filename in os.listdir(directory):
        print filename
        if 'data' in filename:
            dataset = pd.read_csv(os.path.join(directory, filename))
            #print dataset

            last_user_id = 0
            
            for arr in dataset.itertuples():
                user_id = arr[1]
                session_id = arr[3]
                title = arr[4]
                date_time = arr[5].split()
                date = date_time[0]
                #For statistical purposes
                
                if user_id not in user_online_frequency_count:
                    user_online_frequency_count[user_id] = 1
                    last_date = date
                else:
                    if date!=last_date:
                        #print date+" "+last_date
                        user_online_frequency_count[user_id] += 1
                        last_date = date
                        
                        
                month = date.split('-')[1]
                hour = date_time[1].split(":")[0]
                date = date+" "+hour
                played_duration = arr[6]
                action_trigger = arr[7]
                episode_number = arr[9]
                series_total_episodes_count = arr[10]
                if month in month_list: 
                    #print month
                    if last_user_id != user_id:
                        user_action_features[last_user_id] = action_feature
                        user_date_features[last_user_id] = date_feature
                        user_title_features[last_user_id] = title_feature
                        last_user_id = user_id
                        action_feature = {}
                        date_feature = {}
                        title_feature = {}
                        complete_feature = {}
                        last_user_id = user_id

                    #Collect action feature
                    if action_trigger not in action_feature:
                        action_feature[action_trigger] = 1
                    else:
                        action_feature[action_trigger] += 1

                    #Collect date feature
                    #print date
                    if date not in date_feature:
                        date_feature[date] = float(played_duration)
                    else:
                        date_feature[date] += float(played_duration)
                    if date not in date_hour_set:
                        date_hour_set.append(date)
                        



                    #complete = float(episode_number) / float(series_total_episodes_count)
                    #Collect title feature
                    if title not in title_feature:
                        title_feature[title] = 1
                    else:
                        title_feature[title] += 1
                    if title not in title_set:
                        title_set.append(title)
                
            continue
        else:
            continue
    return user_action_features, user_date_features, date_hour_set, user_title_features, title_set, user_online_frequency_count

user_action_features, user_date_features, date_hour_set, user_title_features, title_set, user_online_frequency_count = get_user_features('public')

.DS_Store
data-001.csv
data-002.csv
data-003.csv
data-004.csv
data-005.csv
data-006.csv
data-007.csv
data-008.csv
data-009.csv
data-010.csv
data-011.csv
data-012.csv
data-013.csv
data-014.csv
data-015.csv
data-016.csv
data-017.csv
data-018.csv
data-019.csv
data-020.csv
data-021.csv
data-022.csv
data-023.csv
data-024.csv
data-025.csv
data-026.csv
data-027.csv
data-028.csv
data-029.csv
data-030.csv
data-031.csv
data-032.csv
data-033.csv
data-034.csv
data-035.csv
data-036.csv
data-037.csv
data-038.csv
data-039.csv
data-040.csv
data-041.csv
data-042.csv
data-043.csv
data-044.csv
data-045.csv
data-046.csv
data-047.csv
data-048.csv
data-049.csv
data-050.csv
data-051.csv
data-052.csv
data-053.csv
data-054.csv
data-055.csv
data-056.csv
data-057.csv
data-058.csv
data-059.csv
data-060.csv
data-061.csv
data-062.csv
data-063.csv
data-064.csv
data-065.csv
data-066.csv
data-067.csv
data-068.csv
data-069.csv
data-070.csv
data-071.csv
data-072.csv
data-073.csv
data-074.csv
data-075.csv
label-001.csv
l

In [7]:
def get_user_timeslots(directory):
    user_timeslots = {}
    transitions = []
    for filename in os.listdir(directory):
        #print filename
        if 'label' in filename:
            dataset = pd.read_csv(os.path.join(directory, filename))
            for i in range(0, len(dataset.values)):
                arr = dataset.values[i]
                user_id = arr[0]
                timeslots = []
                for j in range(1, len(arr)):
                    timeslots.append(arr[j])
                user_timeslots[user_id] = timeslots
            continue
        else:
            continue
    return user_timeslots

user_timeslots = get_user_timeslots('public')

In [49]:
def get_train_data(user_action_features, user_date_features, user_timeslots, date_time_set, user_title_features, 
                   title_set, target_index, day, hour_upper_bound, hour_lower_bound):
    X=[]
    y = []
    for user_id in user_timeslots:
        x=[]
        #action
        x = get_user_action_feature(user_id, x, 1)

        #date
        x= get_online_flag_feature(user_id, hour_upper_bound, hour_lower_bound, x, 2)
                

        #week day
        x=get_behavior_feature(user_id, x, 3)
            

        #title
        
        x=get_title_view_feature(user_id, x, 4)
        #generate y
        user_timeslot = user_timeslots[user_id]
        #print x
        X.append(x)
        y.append(user_timeslot[target_index])
    return X, y

#train_X, train_y  = get_train_data(user_action_features, user_date_features, user_timeslots, date_hour_set, user_title_features, completeness_features, title_set, 0)

#print str(train_X)
#print str(train_y)

In [50]:
def get_test_data(user_action_features, user_date_features, user_timeslots, date_time_set, user_title_features,
                  title_set, day, hour_upper_bound, hour_lower_bound):
    X=[]
    
    for user_id in user_id_list:
        
        x = []
        #action
        x = get_user_action_feature(user_id, x, 1)

        #date
        x= get_online_flag_feature(user_id, hour_upper_bound, hour_lower_bound, x, 2)
                

        #week day
        x=get_behavior_feature(user_id, x, 3)
            

        #title
        
        x=get_title_view_feature(user_id, x, 4)
        
        #print x
        X.append(x)

    return X, user_id_list

#test_X, user_id_list= get_test_data(user_action_features, user_date_features, user_timeslots, date_hour_set, user_title_features, completeness_features, title_set, 0, 21, 1)


In [56]:
state = ['limit playzone countdown']
#state = ['limit playzone countdown']

def get_user_action_feature(user_id, x=[], field=1):
    user_feature = {}
    if user_id in user_action_features:
        user_feature = user_action_features[user_id]
        for feature in user_feature:
            if feature in state:
                index = state.index(feature)
                value = 1
                x.append((field, index, value))
                return x
            else:
                x.append((field, 1, 1))
                return x
    return x

In [54]:
def get_online_flag_feature(user_id, hour_upper_bound, hour_lower_bound, x=[], field = 2):
    user_date_feature = {}
    if user_id in user_date_features:
        user_date_feature = user_date_features[user_id]
        online = 0
        for date in user_date_feature:
            day_hour = date.split()
            d = day_hour[0]
            h = int(day_hour[1])

            index =  get_weekday(d)
            if hour_upper_bound != 21:
                if index == day:
                    if (h >= hour_upper_bound) & (h < hour_lower_bound):
                        online += 1
            else:
                if index == day:
                    if (h >= hour_upper_bound) & (h <= 23):
                        online += 1
                elif index == (day+1):
                    if (h >= 0) & (h <= hour_lower_bound):
                        online += 1

        if online > 2:
            x.append((field, 1, 1))
        else:
            x.append((field, 0, 1))
    return x

In [42]:
def get_behavior_feature(user_id, x=[], field = 3):
    user_date_feature = {}
    last_date = ""
    if user_id in user_date_features:
        user_date_feature = user_date_features[user_id]

        day_count = 0
        for date in user_date_feature:
            day_hour = date.split()
            d = day_hour[0]
            h = int(day_hour[1])
            index =  get_weekday(d)
            if index == day:
                if last_date != date:
                    day_count+=1
                    last_date = date
        #print 'day_count: '+str(day_count)
        if day_count > 47:
            x.append((field, 1, 1))
        elif (day_count > 14) & (day_count <= 47):
            x.append((field, 2, 1))
        else:
            x.append((field, 0, 1))
        


    return x

In [43]:
def get_title_view_feature(user_id, x=[], field = 4):
    user_title_feature = {}
    if user_id in user_title_features:
        user_title_feature = user_title_features[user_id]
        field = 4

        if len(user_title_feature) > 12:
            x.append((field, 1, 1))
        elif (len(user_title_feature) > 5) & (len(user_title_feature) <= 12):
            x.append((field, 2, 1))
        else:
            x.append((field, 0, 1))
    return x

In [46]:
def train(i = 22):
    print 'Model '+str(i)
    day = (i+1)/4
    if (i+1)%4 == 0:
        day = ((i+1)/4)-1
    print 'day'+str(day)
    
    hour_upper_bound = 1
    hour_lower_bound = 9
    if ((i+1) % 4) == 0:
        hour_upper_bound = 21
        hour_lower_bound = 1
    elif ((i+1) % 4) == 1:
        hour_upper_bound = 1
        hour_lower_bound = 9
    elif ((i+1) % 4) == 2:
        hour_upper_bound = 9
        hour_lower_bound = 17
    elif ((i+1) % 4) == 1:
        hour_upper_bound = 17
        hour_lower_bound = 21
    train_X, train_y = get_train_data(user_action_features, user_date_features, user_timeslots, date_hour_set, user_title_features, title_set, i, day, hour_upper_bound, hour_lower_bound)
    print(len(train_X), len(train_y))
    print('Training processing!')

    ffm_data = ffm.FFMData(train_X, train_y)


    # train the model for 10 iterations

    n_iter = 100

    model = ffm.FFM(eta=0.1, lam=0.000001, k=20)
    model.init_model(ffm_data)

    for i in range(n_iter):
        model.iteration(ffm_data)

        y_pred = model.predict(ffm_data)
        auc = roc_auc_score(train_y, y_pred)
        if ((i+1) % 10) == 0:
            print("Epoch = %d, train accuracy = %.2f%%" % (i + 1, 100. * auc))


    print('\nTest validation!')

    test_X, user_id_list= get_test_data(user_action_features, user_date_features, user_timeslots, date_hour_set, user_title_features, title_set, day, hour_upper_bound, hour_lower_bound)
    test_y = [0]*len(test_X)
    ffm_test_data = ffm.FFMData(test_X, test_y)
    y_pred = model.predict(ffm_test_data)
    
    # save the model 
    model.save_model('ololo.bin')

    # load it to reuse the model
    model = ffm.read_model('ololo.bin')
    return y_pred

#y_pred = train(22)
#print y_pred

In [55]:
# prepare the data
# (field, index, value) format
predict_list = []
for i in range(0, 28):
    y_pred = train(i)
    print str(y_pred)
    predict_list.append(y_pred)
    #auc = roc_auc_score(test_y, y_pred)
    #print('train auc %.4f' % auc)

    

Model 0
day0
(57159, 57159)
Training processing!
Epoch = 10, train accuracy = 76.50%
Epoch = 20, train accuracy = 76.56%
Epoch = 30, train accuracy = 76.59%
Epoch = 40, train accuracy = 76.59%
Epoch = 50, train accuracy = 76.62%
Epoch = 60, train accuracy = 76.67%
Epoch = 70, train accuracy = 76.71%
Epoch = 80, train accuracy = 76.75%
Epoch = 90, train accuracy = 76.77%
Epoch = 100, train accuracy = 76.75%

Test validation!
[0.26254386 0.12300739 0.00535545 ... 0.06100358 0.09020473 0.5       ]
Model 1
day0
(57159, 57159)
Training processing!
Epoch = 10, train accuracy = 77.61%
Epoch = 20, train accuracy = 77.68%
Epoch = 30, train accuracy = 77.64%
Epoch = 40, train accuracy = 77.72%
Epoch = 50, train accuracy = 77.68%
Epoch = 60, train accuracy = 77.79%
Epoch = 70, train accuracy = 77.70%
Epoch = 80, train accuracy = 77.75%
Epoch = 90, train accuracy = 77.70%
Epoch = 100, train accuracy = 77.77%

Test validation!
[0.38657936 0.18253419 0.00822911 ... 0.09624071 0.12791528 0.5       ]


KeyboardInterrupt: 

In [139]:
def output(output_file, user_id_list, predict_list):
    
    with open(output_file, 'w') as f:
        f.write('user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,time_slot_9,time_slot_10,time_slot_11,time_slot_12,time_slot_13,time_slot_14,time_slot_15,time_slot_16,time_slot_17,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27\n')
        for i in range(0, len(user_id_list)):
            f.write(str(user_id_list[i]))
            for predict_y in predict_list:
                pred_y = 0
                if predict_y[i] != 'nan':
                    pred_y = predict_y[i]
                    
                f.write(","+str(pred_y))
            f.write("\n")
            
    f.close

output('kkstream/predict2.csv', user_id_list, predict_list)

In [3]:
import pandas as pd

df1 = pd.read_csv('kkstream/predict.csv')
source = {}
users = []
for arr in df1.itertuples():
    users.append(arr[1])
    str1 = ""
    for i in range(2, len(arr)):
        str1+=","+str(arr[i])
    source[arr[1]] = str1

dst = {}
df2 = pd.read_csv('kkstream/predict-time.csv')
for arr in df2.itertuples():
    str2 = ""
    for i in range(2, len(arr)):
        str2+=","+str(arr[i])
    dst[arr[1]] = str2

with open('predict_time.csv', 'w') as f:
    f.write('user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,time_slot_9,time_slot_10,time_slot_11,time_slot_12,time_slot_13,time_slot_14,time_slot_15,time_slot_16,time_slot_17,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27\n')
        
    for user in users:
        if user in dst:
            f.write(str(user)+dst[user]+"\n")
        else:
            f.write(str(user)+source[user]+"\n")
f.close

<function close>

In [17]:
day = '2017-06-01'

week_days = [[], [], [], [], [], [], []]

week_count = 3
for month in range(6, 9):
    for day in range(1, 32):
        p_day = '0'
        if day < 10:
            p_day = '0'+str(day)
        else:
            p_day = str(day)
        if (day == 31) & (month == 6):
            continue
        else:
            #print week_count
            #print '2017-0'+str(month)+'-'+p_day
            week_days[week_count].append('2017-0'+str(month)+'-'+p_day)
            
            if week_count < 6:
                week_count +=1
            else:
                week_count = 0

for i in range(0, len(week_days)):
    print (i+1)
    print str(week_days[i])


1
['2017-06-05', '2017-06-12', '2017-06-19', '2017-06-26', '2017-07-03', '2017-07-10', '2017-07-17', '2017-07-24', '2017-07-31', '2017-08-07', '2017-08-14', '2017-08-21', '2017-08-28']
2
['2017-06-06', '2017-06-13', '2017-06-20', '2017-06-27', '2017-07-04', '2017-07-11', '2017-07-18', '2017-07-25', '2017-08-01', '2017-08-08', '2017-08-15', '2017-08-22', '2017-08-29']
3
['2017-06-07', '2017-06-14', '2017-06-21', '2017-06-28', '2017-07-05', '2017-07-12', '2017-07-19', '2017-07-26', '2017-08-02', '2017-08-09', '2017-08-16', '2017-08-23', '2017-08-30']
4
['2017-06-01', '2017-06-08', '2017-06-15', '2017-06-22', '2017-06-29', '2017-07-06', '2017-07-13', '2017-07-20', '2017-07-27', '2017-08-03', '2017-08-10', '2017-08-17', '2017-08-24', '2017-08-31']
5
['2017-06-02', '2017-06-09', '2017-06-16', '2017-06-23', '2017-06-30', '2017-07-07', '2017-07-14', '2017-07-21', '2017-07-28', '2017-08-04', '2017-08-11', '2017-08-18', '2017-08-25']
6
['2017-06-03', '2017-06-10', '2017-06-17', '2017-06-24', '2

In [18]:
def get_weekday(date):
    for i in range(0, len(week_days)):
        week = week_days[i]
        if date in week:
            return i
    return 0

i = get_weekday('2017-06-15')
print i

3


In [18]:
def get_title_view_list(directory):

    title_view_list = {}
   
    for filename in os.listdir(directory):
        #print filename
        if 'data' in filename:
            dataset = pd.read_csv(os.path.join(directory, filename))
            #print dataset

            last_user_id = 0
            for arr in dataset.itertuples():
                user_id = arr[1]
                title = arr[4]

                if last_user_id != user_id:
                    if title in title_view_list:
                        title_view_list[title] += 1
                    else:
                        title_view_list[title] = 1
                    last_user_id = user_id
                
            continue
        else:
            continue
    return title_view_list

title_view_list  = get_title_view_list('public')


TypeError: 'numpy.int64' object is not iterable

In [27]:
list = []
for user_id in user_online_frequency_count:
    if user_id < 57159:
        list.append(user_online_frequency_count[user_id])
        #print (user_online_frequency_count[user_id])
   
list.sort()
print list[int(0.25 * len(list))]
print list[int(0.5 * len(list))]
print list[int(0.75 * len(list))]

4
14
47


In [36]:
list = []
for user_id in user_title_features:
    if user_id < 57159:
        user_title_feature = user_title_features[user_id]
        list.append(len(user_title_feature))

list.sort()
print list[int(0.25 * len(list))]
print list[int(0.5 * len(list))]
print list[int(0.75 * len(list))]

2
6
12


In [9]:
state = ['pause', 'leave', 'next episode', 'interrupt', 'limit playzone countdown', 'seek', 'video ended', 'program stopped or enlarged-reduced', 'program stopped']

list ={}
for user_id in user_action_features:
    #print user_id
    user_action_feature = user_action_features[user_id]
    for feature in user_action_feature:
        if feature in list:
            list[feature]+=user_action_feature[feature]
        else:
            list[feature]=user_action_feature[feature]

    #list.append(len(user_action_feature))

print list

{'network error': 27026, 'program stopped or enlarged-reduced': 43818, 'pause': 3909319, 'cast': 37290, 'program stopped': 95285, 'api error': 512, 'limit single device playing': 2, 'leave': 505239, 'next episode': 387214, 'error': 6199, 'interrupt': 1202601, 'player error': 1626, 'limit playzone countdown': 33558, 'seek': 13800260, 'video ended': 743541}
